In [2]:
import os
from google import genai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

client  = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))

# Talking to Gemini

In [11]:
def get_completion(prompt):
   response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
    )
   return response.text

In [6]:
prompt = "Explain how AI works"
get_completion(prompt)

'Okay, let\'s break down how AI works. It\'s a broad field, so I\'ll cover the core concepts and then dive into some common techniques.\n\n**What is AI? (At its core)**\n\nAt its most fundamental, Artificial Intelligence (AI) is about creating computer systems that can perform tasks that typically require human intelligence. These tasks might include:\n\n*   **Learning:**  Acquiring information and rules for using the information.\n*   **Reasoning:**  Applying logic and making inferences.\n*   **Problem-solving:**  Finding solutions to complex situations.\n*   **Perception:**  Understanding and interpreting sensory data (like images, sound, text).\n*   **Understanding Natural Language:**  Comprehending and generating human language.\n*   **Decision Making:** Choosing between options to achieve a goal.\n\n**The Key Ingredient: Algorithms and Data**\n\nAI systems achieve these abilities by relying heavily on two key ingredients:\n\n1.  **Algorithms:** These are sets of rules or instructi

# Generating Structured Output

In [8]:
prompt="""List a few popular salad recipes in JSON format.

Use this JSON schema:

Recipe: {'recpie_name': 'string', 'ingredients': ['string'], 'instructions': ['string']}
Return: list[Recipe]
"""

print(get_completion(prompt))

```json
[
  {
    "recpie_name": "Classic Caesar Salad",
    "ingredients": [
      "Romaine lettuce",
      "Croutons",
      "Parmesan cheese",
      "Caesar dressing (mayonnaise, garlic, lemon juice, Worcestershire sauce, anchovy paste, Dijon mustard, olive oil, salt, pepper)"
    ],
    "instructions": [
      "Wash and chop the romaine lettuce.",
      "Prepare or buy Caesar dressing.",
      "Combine lettuce, croutons, and parmesan cheese in a bowl.",
      "Toss with Caesar dressing.",
      "Serve immediately."
    ]
  },
  {
    "recpie_name": "Caprese Salad",
    "ingredients": [
      "Fresh mozzarella cheese",
      "Ripe tomatoes",
      "Fresh basil leaves",
      "Balsamic glaze",
      "Olive oil",
      "Salt",
      "Pepper"
    ],
    "instructions": [
      "Slice the mozzarella and tomatoes into roughly equal-sized rounds.",
      "Arrange the mozzarella, tomatoes, and basil leaves on a platter, alternating them.",
      "Drizzle with balsamic glaze and olive oil."

## Using Pydantic to Generate Structured Output

In [21]:
from pydantic import BaseModel

class Recipe(BaseModel):
    recipe_name: str
    ingredients: list[str]
    instructions: list[str]


In [22]:
def get_completion_with_config(prompt, config):
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        config=config,
    )
    return response

In [23]:
config = {
    'response_mime_type': 'application/json',
    'response_schema': list[Recipe]
}

In [19]:
response = get_completion_with_config(prompt, config)
print(response.text)

[
  {
    "recipe_name": "Caesar Salad",
    "ingredients": ["Romaine lettuce", "Croutons", "Parmesan cheese", "Caesar dressing (mayonnaise, garlic, lemon juice, Worcestershire sauce, anchovies, black pepper)"],
    "instructions": ["Wash and chop romaine lettuce.", "Prepare or buy croutons.", "Grate parmesan cheese.", "Make or buy Caesar dressing.", "Combine all ingredients and toss well.", "Serve immediately"]
  },
  {
    "recipe_name": "Greek Salad",
    "ingredients": ["Tomatoes", "Cucumbers", "Red onion", "Kalamata olives", "Feta cheese", "Olive oil", "Red wine vinegar", "Oregano", "Salt", "Pepper"],
    "instructions": ["Chop tomatoes and cucumbers.", "Slice red onion thinly.", "Combine tomatoes, cucumbers, red onion, and olives in a bowl.", "Add feta cheese.", "Dress with olive oil, red wine vinegar, oregano, salt, and pepper.", "Toss gently and serve." ]
  },
  {
    "recipe_name": "Caprese Salad",
    "ingredients": ["Fresh mozzarella", "Tomatoes", "Fresh basil", "Balsamic gl

### Parsing the Structured Output into Objects

In [24]:
my_recipes: list[Recipe] = response.parsed

## Using Enum to Constrain Output

In [25]:
import enum

In [26]:
class Instrument(enum.Enum):
    PERCUSSION = "Percussion"
    STRING = "String"
    WOODWIND = "Woodwind"
    BRASS = "Brass"
    KEYBOARD = "Keyboard"

In [27]:
config = {
    'response_mime_type': 'text/x.enum',
    'response_schema': Instrument
}

In [29]:
prompt = "'What type of instrument is an oboe?"

response = get_completion_with_config(prompt, config)

In [30]:
print(response.text)

Woodwind


### Passing the Schema as JSON

In [ ]:
config = {
    'response_mime_type': 'text/x.enum',
    'response_schema': {
        "type": "STRING",
        "enum": ["Percussion", "String", "Woodwind", "Brass", "Keyboard"], 
    },
}

In [32]:
response = get_completion_with_config(prompt, config)
print(response.text)

Woodwind


In [33]:
class Grade(enum.Enum):
    A_PLUS = "A+"
    A = "A"
    B = "B"
    C = "C"
    D = "D"
    F = "F"

In [35]:
class RatededRecipe(BaseModel):
    recipe_name: str
    rating: Grade
    ingredients: list[str]
    instructions: list[str]

In [40]:
config = {
    'response_mime_type': 'application/json',
    'response_schema': list[RatededRecipe],
}

In [41]:
prompt = 'List 10 salad recipes and give them grades based on tastiness.'
response = get_completion_with_config(prompt, config)
print(response.text)

[
  {
    "recipe_name": "Classic Caesar Salad",
    "rating": "A",
    "ingredients": ["Romaine lettuce", "Croutons", "Parmesan cheese", "Caesar dressing"],
    "instructions": ["Wash and chop romaine lettuce.", "Prepare or buy croutons.", "Grate Parmesan cheese.", "Toss lettuce, croutons, and cheese with Caesar dressing.", "Serve immediately"]
  },
  {
    "recipe_name": "Greek Salad",
    "rating": "A",
    "ingredients": ["Cucumber", "Tomato", "Red onion", "Kalamata olives", "Feta cheese", "Olive oil", "Red wine vinegar", "Oregano"],
    "instructions": ["Chop cucumber, tomato, and red onion.", "Combine with olives and feta cheese.", "Dress with olive oil, red wine vinegar, and oregano.", "Toss gently and serve"]
  },
  {
    "recipe_name": "Caprese Salad",
    "rating": "A",
    "ingredients": ["Fresh mozzarella", "Tomatoes", "Fresh basil", "Balsamic glaze", "Olive oil"],
    "instructions": ["Slice mozzarella and tomatoes.", "Arrange mozzarella, tomatoes, and basil leaves on a pl